In [1]:
%load_ext autoreload
%autoreload 2
%run ../src/notebook_env.py

In [2]:
%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import Dataset
import torch.nn as nn

from copy import deepcopy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split

In [3]:
# import sys
# sys.path.append('/home/jcejudo/rd-img-classification-pilot/src')
from torch_utils import *
from ds_utils import *
from gradcam import *
from models import *

In [4]:
root_results_path = '/home/jcejudo/rd-img-classification-pilot/results'

experiment_name = 'new_implementation'

check_path(root_results_path)
results_path = os.path.join(root_results_path,experiment_name)
check_path(results_path)

In [5]:
#get data
data_path = '/home/jcejudo/rd-img-classification-pilot/training_data/ec'
ec_df = path2DataFrame(data_path)

data_path = '/home/jcejudo/rd-img-classification-pilot/training_data/getty'
getty_df = path2DataFrame(data_path)

df = pd.concat((ec_df,getty_df))

X = df['file_path'].values
y = df['category'].values

y_encoded, encoding_dict = label_encoding(y)


In [6]:
class TrainingDataset(Dataset):
    """
    Pytorch training dataset class
    X: Numpy array containing the paths to the images
    y: Numpy array with the encoded labels
    """
    def __init__(self, X, y, transform=None):
        self.transform = transform
        self.X = X
        self.y = y        
        self.N = y.shape[0]   

    def __len__(self):
        return self.N

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.X[idx]
        img = Image.open(img_path).convert('RGB')
        label = self.y[idx]
                
        if self.transform:
            img = self.transform(img)

        return img,label
    
base_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # this normalization is required https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.33)

traindata = TrainingDataset(X_train,y_train,transform=base_transform)
trainloader = torch.utils.data.DataLoader(traindata, batch_size=32,shuffle=True, num_workers=2)

testdata = TrainingDataset(X_test,y_test,transform=base_transform)
testloader = torch.utils.data.DataLoader(testdata, batch_size=32,shuffle=True, num_workers=2)

In [7]:
import torch
device, _ = check_cuda()
#model_path = '/home/jcejudo/rd-img-classification-pilot/results/XAI/split_0/checkpoint.pth'
net = ResNet(34,20)
net = net.to(device)

Using a single device 



In [8]:
def evaluate(dataloader,confusion_matrix = False):
  #accuracy
  correct = 0
  total = 0
  #confusion matrix
  cm = None

  i = 0

  if confusion_matrix:
    ground_truth_list = []
    predictions_list = []

  with torch.no_grad():
      for data in dataloader:
          #print(i)
          images, labels = data
        
          images = images.to(device)
          labels = labels.to(device)
        
          outputs = net(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
        
          i += 1

          if confusion_matrix:
            ground_truth_list += deepcopy(labels)
            predictions_list += list(predicted.cpu())

  if confusion_matrix:
    cm = sklearn.metrics.confusion_matrix(ground_truth_list,predictions_list,labels=np.arange(20))

  return correct/total, cm


In [11]:
#net = ConvNet(3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        #print(i)
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        loss = criterion(net(inputs), labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    running_loss /= i
        
    
        #running_loss = 0.0
        #if i % 100 == 99: 
    print('evaluating...')
    acc,_ = evaluate(testloader)
    print('[%d, %5d] loss: %.3f   test acc: %.3f' %
          (epoch + 1, i + 1, running_loss ,acc))
            
            
torch.save(net.state_dict(),os.path.join(results_path,'checkpoint.pth'))

print('Finished Training')

/home/jcejudo/.local/lib/python3.8/site-packages/PIL/Image.py:2847: DecompressionBombWarning: Image size (149839008 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[1,   390] loss: 2.316   test acc: 0.760


/home/jcejudo/.local/lib/python3.8/site-packages/PIL/Image.py:2847: DecompressionBombWarning: Image size (149839008 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[2,   390] loss: 2.265   test acc: 0.778
Finished Training


In [12]:
acc,confusion_matrix = evaluate(testloader,confusion_matrix=True)

ValueError: Classification metrics can't handle a mix of unknown and multiclass targets

In [ ]:
f'accuracy on test data: {acc}'

In [ ]:
confusion_matrix